# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 05: Batch Predictions</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/credit_scores/5_batch_predictions.ipynb)


## 🗒️ This notebook is divided into the following sections:

1. Load batch data
2. Predict using model from Model Registry

![part3](../../images/03_model.png) 

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> ✨ Loading Batch Data </span>

In [2]:
feature_view = fs.get_feature_view(
    name="credit_scores",
    version=1
)

feature_view.init_batch_scoring(
    training_dataset_version=1
)

df = feature_view.get_batch_data()
df.head()

2023-03-05 09:29:28,280 INFO: USE `dowlingj_featurestore`
2023-03-05 09:29:28,887 INFO: SELECT `fg7`.`credit_active` `credit_active`, `fg7`.`credit_currency` `credit_currency`, `fg7`.`days_credit` `days_credit`, `fg7`.`credit_day_overdue` `credit_day_overdue`, `fg7`.`days_credit_enddate` `days_credit_enddate`, `fg7`.`cnt_credit_prolong` `cnt_credit_prolong`, `fg7`.`amt_credit_sum` `amt_credit_sum`, `fg7`.`amt_credit_sum_debt` `amt_credit_sum_debt`, `fg7`.`amt_credit_sum_overdue` `amt_credit_sum_overdue`, `fg7`.`credit_type` `credit_type`, `fg7`.`days_credit_update` `days_credit_update`, `fg0`.`name_contract_type` `name_contract_type`, `fg0`.`code_gender` `code_gender`, `fg0`.`flag_own_car` `flag_own_car`, `fg0`.`flag_own_realty` `flag_own_realty`, `fg0`.`cnt_children` `cnt_children`, `fg0`.`amt_income_total` `amt_income_total`, `fg0`.`amt_annuity` `amt_annuity`, `fg0`.`amt_goods_price` `amt_goods_price`, `fg0`.`name_type_suite` `name_type_suite`, `fg0`.`name_income_type` `name_income_t

,credit_active,credit_currency,days_credit,credit_day_overdue,days_credit_enddate,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_overdue,credit_type,...,cnt_drawings_atm_current,cnt_drawings_current,cnt_drawings_other_current,cnt_drawings_pos_current,cnt_instalment_mature_cum,name_contract_status,sk_dpd,sk_dpd_def,sk_id_curr,previous_loan_counts
0,0,0,0.816761,0.0,0.459338,0.0,-0.687989,-0.606573,0.0,2,...,-0.349365,-0.317357,0.0,-0.297573,-0.608684,1,-0.21157,-0.21157,-0.616544,-1.514632
1,0,0,0.816761,0.0,0.459338,0.0,-0.687989,-0.606573,0.0,2,...,-0.349365,-0.317357,0.0,-0.297573,-0.608684,1,-0.21157,-0.21157,-0.616544,-1.514632
2,0,0,0.816761,0.0,0.459338,0.0,-0.687989,-0.606573,0.0,2,...,-0.349365,-0.317357,0.0,-0.297573,-0.608684,1,-0.21157,-0.21157,-0.616544,-1.514632
3,0,0,0.816761,0.0,0.459338,0.0,-0.687989,-0.606573,0.0,2,...,-0.349365,-0.317357,0.0,-0.297573,-0.608684,1,-0.21157,-0.21157,-0.616544,-1.514632
4,0,0,0.816761,0.0,0.459338,0.0,-0.687989,-0.606573,0.0,2,...,-0.349365,-0.317357,0.0,-0.297573,-0.608684,1,-0.21157,-0.21157,-0.616544,-1.514632


In [ ]:
def get_cols_with_missing_values(df):
    cols = []
    for item in df.columns:Q
        if df[item].isna().sum() > 0:
            cols.append(item)        
    return cols

cols = get_cols_with_missing_values(df)
df = df.drop(columns=cols)

## <span style="color:#ff5f27;"> 🗄 Retrieving model from Model Registry </span>

In [3]:
import joblib

model = mr.get_model(
    name="credit_scores_model",
    version=1
)
model_dir = model.download()
model = joblib.load(model_dir + "/credit_scores_model.pkl")
model

RandomForestClassifier(class_weight={0: 0.09999999999999998, 1: 0.9},
                       n_estimators=25, n_jobs=-1, random_state=42)

## <span style="color:#ff5f27;"> 🤖 Making the predictions </span>

In [4]:
predictions = model.predict(df)
predictions[:10]

Feature names unseen at fit time:
- amt_credit_sum_overdue
- amt_drawings_other_current
- cnt_credit_prolong
- cnt_drawings_other_current
- credit_day_overdue
- ...



ValueError: X has 118 features, but RandomForestClassifier is expecting 104 features as input.

---